In [45]:
import sys
print(sys.executable)

C:\Users\DFS\Desktop\gitrepo\env_arm64\Scripts\python.exe


In [46]:
# Let's check to ensure we are in the proper environment, remember we need to be in pure arm64
import platform

arch = platform.machine()
sys = platform.system()
processor = platform.processor()
print(f"{arch}\n{sys}\n{processor}")

ARM64
Windows
ARMv8 (64-bit) Family 8 Model 1 Revision 201, Qualcomm Technologies Inc


In [47]:
# Necessary tools that we need
import onnxruntime as ort
import os
import numpy as np
import time

from pathlib import Path
from tokenizers import Tokenizer

In [48]:
# Grab the root directory as a reference
root_dir = Path.cwd().parent.parent
root_dir

WindowsPath('C:/Users/DFS/Desktop/gitrepo/qnn_sample_apps')

In [49]:
# Grab the path to onnxruntime therefore we can grab hexagon driver
onnx_root = Path(ort.__file__).parent
onnx_root

WindowsPath('C:/Users/DFS/Desktop/gitrepo/env_arm64/Lib/site-packages/onnxruntime')

In [50]:
# Subdirectory where all .onnx dependencies are located
model_subdirectory = "mistral-7b-instruct-v0.2-cpu-int4-rtn-block-32-acc-level-4"

# The embeddings model is entry point, use netron to visualize
model_name = "mistral-7b-instruct-v0.2-cpu-int4-rtn-block-32-acc-level-4.onnx"

# Tokenizer
tokenizer_json = "tokenizer.json"

In [51]:
# Solidifying all paths

model_path = root_dir/"models"/model_subdirectory/model_name
tokenizer_path = root_dir/"models"/model_subdirectory/tokenizer_json

In [52]:
model_path.is_file()

True

In [53]:
session_options = ort.SessionOptions()

# Creating an inference session for the embedding graph
session = ort.InferenceSession(model_path)

session.get_providers()

['CPUExecutionProvider']

In [54]:
inputs = session.get_inputs()
outputs = session.get_outputs()

In [55]:
for layer in inputs:
    print(f"Name: {layer.name}\n\tExpected Input Shape: {layer.shape}\n\tExpected Input Type: {layer.type}")
    print("*"*100)

Name: input_ids
	Expected Input Shape: ['batch_size', 'sequence_length']
	Expected Input Type: tensor(int64)
****************************************************************************************************
Name: attention_mask
	Expected Input Shape: ['batch_size', 'total_sequence_length']
	Expected Input Type: tensor(int64)
****************************************************************************************************
Name: past_key_values.0.key
	Expected Input Shape: ['batch_size', 8, 'past_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: past_key_values.0.value
	Expected Input Shape: ['batch_size', 8, 'past_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: past_key_values.1.key
	Expected Input Shape: ['batch_size', 8, 'past_sequence_length', 128]

In [56]:
for layer in outputs:
    print(f"Name: {layer.name}\n\tExpected Input Shape: {layer.shape}\n\tExpected Input Type: {layer.type}")
    print("*"*100)

Name: logits
	Expected Input Shape: ['batch_size', 'sequence_length', 32000]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.0.key
	Expected Input Shape: ['batch_size', 8, 'total_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.0.value
	Expected Input Shape: ['batch_size', 8, 'total_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.1.key
	Expected Input Shape: ['batch_size', 8, 'total_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.1.value
	Expected Input Shape: ['batch_size', 8, 'total_sequence_length', 128]
	Expect

In [57]:
# Load in tokenizer using tokenizer path above
tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [58]:
# An initial query
init_query = "[INST]\nYou are a very experienced Yoga instructor.\nProvide me with an yoga routine to mitigate lower back pain. Keep response concise and only provide the routine and provide in a json format for easy parsing. Example {\
 'name': 'Cat Cow Pose',\
 'description': 'Gently moving the spine from flexion to extension, opening up the chest and releasing tension in the back.',\
 'instructions': 'Start on all fours with wrists under shoulders and knees under hips. Inhale as you arch your back and lift your head and tailbone, exhale as you round your spine and tuck your chin towards your chest.'\
 }\n[/INST]</s>"
encoding = tokenizer.encode(init_query)

In [59]:
print("Token IDs:", encoding.ids)
print("Tokens:", encoding.tokens)

Token IDs: [1, 733, 16289, 28793, 13, 1976, 460, 264, 1215, 8304, 627, 10491, 27679, 28723, 13, 18325, 547, 528, 395, 396, 21615, 11935, 298, 2367, 11590, 3889, 852, 3358, 28723, 10339, 2899, 3078, 864, 304, 865, 3084, 272, 11935, 304, 3084, 297, 264, 7611, 5032, 354, 3411, 940, 11601, 28723, 16693, 371, 464, 861, 1869, 464, 19962, 19283, 367, 645, 647, 464, 6518, 1869, 464, 28777, 2250, 5272, 272, 24496, 477, 7584, 296, 298, 8223, 28725, 7032, 582, 272, 8118, 304, 28364, 15802, 297, 272, 852, 13007, 464, 4138, 8373, 1869, 464, 4130, 356, 544, 285, 2020, 395, 1425, 1583, 916, 10693, 304, 13431, 916, 25760, 28723, 560, 28716, 883, 390, 368, 4219, 574, 852, 304, 7818, 574, 1335, 304, 8675, 15600, 28725, 26112, 883, 390, 368, 3713, 574, 24496, 304, 261, 1384, 574, 17418, 5083, 574, 8118, 1815, 443, 13, 28792, 28748, 16289, 28793, 2]
Tokens: ['<s>', '▁[', 'INST', ']', '<0x0A>', 'You', '▁are', '▁a', '▁very', '▁experienced', '▁Y', 'oga', '▁instructor', '.', '<0x0A>', 'Prov', 'ide', '▁me', '▁

In [60]:
input_ids = encoding.ids
input_ids

[1,
 733,
 16289,
 28793,
 13,
 1976,
 460,
 264,
 1215,
 8304,
 627,
 10491,
 27679,
 28723,
 13,
 18325,
 547,
 528,
 395,
 396,
 21615,
 11935,
 298,
 2367,
 11590,
 3889,
 852,
 3358,
 28723,
 10339,
 2899,
 3078,
 864,
 304,
 865,
 3084,
 272,
 11935,
 304,
 3084,
 297,
 264,
 7611,
 5032,
 354,
 3411,
 940,
 11601,
 28723,
 16693,
 371,
 464,
 861,
 1869,
 464,
 19962,
 19283,
 367,
 645,
 647,
 464,
 6518,
 1869,
 464,
 28777,
 2250,
 5272,
 272,
 24496,
 477,
 7584,
 296,
 298,
 8223,
 28725,
 7032,
 582,
 272,
 8118,
 304,
 28364,
 15802,
 297,
 272,
 852,
 13007,
 464,
 4138,
 8373,
 1869,
 464,
 4130,
 356,
 544,
 285,
 2020,
 395,
 1425,
 1583,
 916,
 10693,
 304,
 13431,
 916,
 25760,
 28723,
 560,
 28716,
 883,
 390,
 368,
 4219,
 574,
 852,
 304,
 7818,
 574,
 1335,
 304,
 8675,
 15600,
 28725,
 26112,
 883,
 390,
 368,
 3713,
 574,
 24496,
 304,
 261,
 1384,
 574,
 17418,
 5083,
 574,
 8118,
 1815,
 443,
 13,
 28792,
 28748,
 16289,
 28793,
 2]

In [61]:
# Preparing inputs for prompt

# Number of input sequences processed simultaneously
batch_size = 1

# Current sequence length for initial prompt (number of tokens in current sequence)
seq_len = len(input_ids)#.shape[2]

# Dimensionality of each token embedding vector
# hidden_size = embedding_output.shape[2]

# Number of attention heads in each transformer layer
num_heads = 32

# Size of each attention head (should be hidden_size // num_heads
attn_head_size = 128 #hidden_size//num_heads # ex. 1536/12 = 128

# Total number of transformer layers
num_layers = 32

# This is not the model's global context window (131072), this is the max number of tokens passed in the first forward pass
max_seq_len = len(input_ids)

# Sampling temperature for softmax-based logit scaling
temp = 0.9

# Number of key/value heads (key/value heads are shared amongst attention heads)
num_key_value_heads = 8

In [62]:
len(input_ids)

145

In [63]:
# pad the inputs to expected size of seq_len of 64
# target_seq_len = 64
# input_ids += [pad_token_id] * (target_seq_len - len(input_ids))
input_ids = np.array([input_ids], dtype=np.int64)
input_ids.shape

(1, 145)

In [64]:
attention_mask = np.ones((batch_size, max_seq_len), dtype=np.int64)
attention_mask

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [65]:
attention_mask.shape

(1, 145)

In [66]:
# Let's initialize our KV cache for all transformer layers
empty_kv = {}
for i in range(num_layers):
    # Shape of key and value tensors for each transformer layer
    past_shape = (batch_size, num_key_value_heads, max_seq_len, attn_head_size)

    # Initialize past keys for layer i (used in attention mechanism to avoid recomputation
    empty_kv[f"past_key_values.{i}.key"] = np.zeros(past_shape, dtype=np.float32)

    # Initialize past values for layer i
    empty_kv[f"past_key_values.{i}.value"] = np.zeros(past_shape, dtype=np.float32)

len(empty_kv)

64

In [67]:
empty_kv.keys()

dict_keys(['past_key_values.0.key', 'past_key_values.0.value', 'past_key_values.1.key', 'past_key_values.1.value', 'past_key_values.2.key', 'past_key_values.2.value', 'past_key_values.3.key', 'past_key_values.3.value', 'past_key_values.4.key', 'past_key_values.4.value', 'past_key_values.5.key', 'past_key_values.5.value', 'past_key_values.6.key', 'past_key_values.6.value', 'past_key_values.7.key', 'past_key_values.7.value', 'past_key_values.8.key', 'past_key_values.8.value', 'past_key_values.9.key', 'past_key_values.9.value', 'past_key_values.10.key', 'past_key_values.10.value', 'past_key_values.11.key', 'past_key_values.11.value', 'past_key_values.12.key', 'past_key_values.12.value', 'past_key_values.13.key', 'past_key_values.13.value', 'past_key_values.14.key', 'past_key_values.14.value', 'past_key_values.15.key', 'past_key_values.15.value', 'past_key_values.16.key', 'past_key_values.16.value', 'past_key_values.17.key', 'past_key_values.17.value', 'past_key_values.18.key', 'past_key_v

In [68]:
init_prompt_inputs = {
    "input_ids": input_ids,
    "attention_mask":attention_mask,
    **empty_kv,
}
init_prompt_inputs

{'input_ids': array([[    1,   733, 16289, 28793,    13,  1976,   460,   264,  1215,
          8304,   627, 10491, 27679, 28723,    13, 18325,   547,   528,
           395,   396, 21615, 11935,   298,  2367, 11590,  3889,   852,
          3358, 28723, 10339,  2899,  3078,   864,   304,   865,  3084,
           272, 11935,   304,  3084,   297,   264,  7611,  5032,   354,
          3411,   940, 11601, 28723, 16693,   371,   464,   861,  1869,
           464, 19962, 19283,   367,   645,   647,   464,  6518,  1869,
           464, 28777,  2250,  5272,   272, 24496,   477,  7584,   296,
           298,  8223, 28725,  7032,   582,   272,  8118,   304, 28364,
         15802,   297,   272,   852, 13007,   464,  4138,  8373,  1869,
           464,  4130,   356,   544,   285,  2020,   395,  1425,  1583,
           916, 10693,   304, 13431,   916, 25760, 28723,   560, 28716,
           883,   390,   368,  4219,   574,   852,   304,  7818,   574,
          1335,   304,  8675, 15600, 28725, 26112, 

In [69]:
init_prompt_inputs.get("past_key_values.0.key").shape

(1, 8, 145, 128)

In [70]:
# Run embedding session first
session_output = session.run(None, init_prompt_inputs)
# print("Logits:\n(batch, sequence length, vocab size)")
session_output[0].shape

(1, 145, 32000)

In [71]:
print("Logits:\n(batch, sequence length, vocab size)")
session_output[0].shape

Logits:
(batch, sequence length, vocab size)


(1, 145, 32000)

In [72]:
print("KV Cache:\n(batch, num_kv_heads, sequence length, attn_head_size)")
session_output[1].shape

KV Cache:
(batch, num_kv_heads, sequence length, attn_head_size)


(1, 8, 145, 128)

### To get longer initial context run ctx session over multiple prompts BUT use updated key/values after each prompt

In [73]:
# Update kv cache
present_kv = {f"past_key_values.{i}.key": session_output[1 + i * 2] for i in range(num_layers)}
present_kv.update({f"past_key_values.{i}.value": session_output[1 + i * 2 + 1] for i in range(num_layers)})
present_kv

{'past_key_values.0.key': array([[[[-8.91517177e-02, -2.15142090e-02,  2.59727836e-01, ...,
           -1.31860435e+00, -1.97660625e+00, -2.13837981e+00],
          [ 5.16331434e+00, -3.79646420e+00, -2.57825661e+00, ...,
            2.27529168e+00,  9.08183992e-01,  2.05774516e-01],
          [ 2.90730143e+00, -5.93867493e+00,  1.20966434e-01, ...,
            2.55401802e+00,  1.70817661e+00,  9.30485427e-01],
          ...,
          [-8.40705204e+00, -6.63777018e+00,  8.85535240e-01, ...,
            2.55419946e+00,  1.70737147e+00,  9.30024981e-01],
          [-2.71456003e+00, -1.50452220e+00, -1.18021393e+00, ...,
            1.89126658e+00,  6.98301017e-01, -2.32652724e-01],
          [ 1.83907092e-01,  2.39518255e-01, -7.91722119e-01, ...,
            3.21482033e-01,  2.95929700e-01,  2.85366237e-01]],
 
         [[ 6.77808821e-02,  8.55805725e-02,  4.95823845e-02, ...,
            3.34732842e+00, -3.46340132e+00,  1.77555263e+00],
          [ 2.37771481e-01, -6.58084154e-01, -3

In [74]:
present_kv.keys()

dict_keys(['past_key_values.0.key', 'past_key_values.1.key', 'past_key_values.2.key', 'past_key_values.3.key', 'past_key_values.4.key', 'past_key_values.5.key', 'past_key_values.6.key', 'past_key_values.7.key', 'past_key_values.8.key', 'past_key_values.9.key', 'past_key_values.10.key', 'past_key_values.11.key', 'past_key_values.12.key', 'past_key_values.13.key', 'past_key_values.14.key', 'past_key_values.15.key', 'past_key_values.16.key', 'past_key_values.17.key', 'past_key_values.18.key', 'past_key_values.19.key', 'past_key_values.20.key', 'past_key_values.21.key', 'past_key_values.22.key', 'past_key_values.23.key', 'past_key_values.24.key', 'past_key_values.25.key', 'past_key_values.26.key', 'past_key_values.27.key', 'past_key_values.28.key', 'past_key_values.29.key', 'past_key_values.30.key', 'past_key_values.31.key', 'past_key_values.0.value', 'past_key_values.1.value', 'past_key_values.2.value', 'past_key_values.3.value', 'past_key_values.4.value', 'past_key_values.5.value', 'past

In [75]:
# Dimension checks
present_kv["past_key_values.0.key"].shape

(1, 8, 145, 128)

In [76]:
present_kv["past_key_values.27.value"].shape

(1, 8, 145, 128)

In [77]:
logits = session_output[0]
logits

array([[[-6.2111111e+00, -6.3144040e+00, -1.8914104e-02, ...,
         -4.8969259e+00, -3.6258790e+00, -4.5223360e+00],
        [-8.0700922e+00, -8.0098324e+00, -1.7531357e+00, ...,
         -4.1472197e+00, -3.9645410e+00, -2.3115730e+00],
        [-3.8357506e+00, -4.5648861e+00, -1.4914541e+01, ...,
          1.3736466e-01, -9.2904188e-02,  4.8352626e-01],
        ...,
        [-3.0008442e+00, -5.6037841e+00, -3.9405132e+01, ...,
         -1.9596143e-01,  9.1380495e-01, -2.2220587e-02],
        [-8.9721578e-01, -2.9575696e+00, -4.1037960e+01, ...,
          5.8337694e-01, -1.2999819e-01, -3.8488805e-01],
        [-2.1169434e+00, -1.1676441e+00, -4.5001114e+01, ...,
         -8.6796552e-01,  1.1274946e+00,  2.6438883e-01]]], dtype=float32)

In [78]:
logits[0,-1].shape

(32000,)

In [79]:
def softmax_numpy(x: np.array, temperature: float=1) -> np.array:
    # stabilize x in case of large numbers 
    x = x - np.max(x)

    # Apply temperature
    x = x/temperature

    # Apply Softmax
    return np.exp(x)/np.sum(np.exp(x), axis=-1)

def top_k_probas(probas: np.array, k: int=5) -> np.array:
    # Copy probas so in-place operations don't work on original variable
    probas = probas.copy()
    # Normalize probabilities
    probas /= np.sum(probas)
    # Using -probas to get in descending order
    top_indices_sorted = np.argsort(-probas)[:k]
    top_k_probas = probas[top_indices_sorted]

    # Renormalize top-k probabilites to sum to 1 (probabilites must sum to 1 to use np.random.choice
    top_k_probas /= np.sum(top_k_probas)

    # Return top k probabilities
    return top_indices_sorted, top_k_probas

def apply_repetition_penalty(logits, generated_ids, penalty=1.1):
    for token_id in set(generated_ids):
        logits[token_id] /= penalty
    return logits

In [80]:
# Softmax implemented
# x-np.max(x) => for stability in case of large numbers
softmax = lambda x, temperature=1: np.exp((x-np.max(x))/temperature)/np.sum(np.exp((x-np.max(x))/temperature), axis=-1)

In [81]:
softmax_numpy(logits[0,-1])

array([2.8508414e-06, 7.3662814e-06, 6.7703948e-25, ..., 9.9402459e-06,
       7.3116367e-05, 3.0844134e-05], dtype=float32)

In [82]:
softmax(logits[0,-1])

array([2.8508414e-06, 7.3662814e-06, 6.7703948e-25, ..., 9.9402459e-06,
       7.3116367e-05, 3.0844134e-05], dtype=float32)

In [83]:
# Grabs last tokens logits
temp = 0.6
probas = softmax(logits[0,-1], temperature=temp)
# probas = probas / probas.sum()
next_token_id = int(np.random.choice(len(probas), p=probas)) #int(np.argmax(probas))
next_token_id

733

In [84]:
np.sum(probas)

1.0

In [85]:
tokenizer.decode([next_token_id])

'['

In [86]:
logits.shape

(1, 145, 32000)

In [87]:
temp = 0.6
start = time.time()
max_tokens = 1000
top_k = 50
generated_ids = [next_token_id]
prev_seq_len = logits.shape[1]
printed_length = 0
# print(prev_seq_len)
# print(attention_mask.shape)
print("\nInitial Query:\n", init_query)
print("Generated:")
for _ in range(max_tokens):
    input_ids = np.array([[next_token_id]], dtype=np.int64)
    # print(tokenizer.decode(generated_ids, skip_special_tokens=True))
    # print(tokenizer.decode([next_token_id], skip_special_tokens=False),end=" ")
    
    iter_inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    **present_kv,
    }

    session_output = session.run(None, iter_inputs)
    prev_seq_len += 1
    # Update attention mask
    attention_mask = np.ones((batch_size, prev_seq_len), dtype=np.int64)
    # Update KV Cache
    present_kv = {f"past_key_values.{i}.key": session_output[1 + i * 2] for i in range(num_layers)}
    present_kv.update({f"past_key_values.{i}.value": session_output[1 + i * 2 + 1] for i in range(num_layers)})
    # print(prev_seq_len)
    # print(present_kv.get("past_key_values.0.key").shape)
    # print(len(attention_mask))
    logits = session_output[0]

    token_logits = logits[0,-1]
    token_logits = apply_repetition_penalty(token_logits, generated_ids, penalty=1.1)
#     # Get probabilities
    probas = softmax(token_logits, temperature=temp)
    top_indices, top_probas = top_k_probas(probas, k=top_k) 
    next_token_id = int(np.random.choice(top_indices, p=top_probas)) #int(np.argmax(probas))
    generated_ids.append(next_token_id)
    full_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    new_text = full_text[printed_length:]
    if new_text:
        print(new_text, end="", flush=True)
        printed_length = len(full_text)

    if next_token_id == tokenizer.token_to_id("< | end_of_sentence | >"):
        break
end = time.time()
elapsed = end - start
tps = np.round((max_tokens / elapsed), 2)
print(f"\nTokens Per Second: {tps}")
output_text = tokenizer.decode(generated_ids, skip_special_tokens=True)



Initial Query:
 [INST]
You are a very experienced Yoga instructor.
Provide me with an yoga routine to mitigate lower back pain. Keep response concise and only provide the routine and provide in a json format for easy parsing. Example { 'name': 'Cat Cow Pose', 'description': 'Gently moving the spine from flexion to extension, opening up the chest and releasing tension in the back.', 'instructions': 'Start on all fours with wrists under shoulders and knees under hips. Inhale as you arch your back and lift your head and tailbone, exhale as you round your spine and tuck your chin towards your chest.' }
[/INST]</s>
Generated:
[
  {
    "name": "Cat Cow Pose",
    "description": "Gently moving the spine from flexion to extension, opening up the chest and releasing tension in the back.",
    "instructions": "Start on all fours with wrists under shoulders and knees under hips. Inhale as you arch your back and lift your head and tailbone, exhale as you round your spine and tuck your chin towar

4000/36.66 -> Deepseek R1 MAC M1

In [ ]:
print(output_text)

In [ ]:
json_example_add_to_prompt = {
 "routine": [
 {
 "name": "Cat Cow Pose",
 "description": "Gently moving the spine from flexion to extension, opening up the chest and releasing tension in the back.",
 "instructions": "Start on all fours with wrists under shoulders and knees under hips. Inhale as you arch your back and lift your head and tailbone, exhale as you round your spine and tuck your chin towards your chest."
 },
 {
 "name": "Downward Facing Dog",
 "description": "Stretches the entire back body, calms the mind, and improves digestion.",
 "instructions": "Begin on hands and knees, spread fingers wide and turn toes under. Exhale as you lift your hips up and back, reaching for the legs to be hip-width apart and parallel. Press firmly through the hands and feet."
 },
 {
 "name": "Child's Pose",
 "description": "Relaxes the muscles of the back and neck, allowing for a gentle stretch of the upper body.",
 "instructions": "Kneel on the floor with big toes touching and hips widened. Lower the torso between the thighs, extending arms forward if comfortable or keeping them by the sides. Rest the forehead on the mat, deepening the stretch with each exhale."
 },
 {
 "name": "Bridge Pose",
 "description": "Stretches the chest, neck, and spine while strengthening the back muscles.",
 "instructions": "Lie on your back with knees bent and feet hip-distance apart, arms by your sides. Exhale to press into the feet and lift the hips up towards the ceiling, reaching towards the sky with your hands."
 }]
     }